Below are parameters of the experiments reported in [KLNO24](https://eprint.iacr.org/2024/1972).

In [1]:
def largest_a_below_binom_sum(n, threshold_power):
    """
    Returns the largest a such that sum_{i=0}^a binomial(n, i) < 2^threshold_power

    Parameters:
    - n: total number of bits
    - threshold_power: the exponent k such that the threshold is 2^k

    Returns:
    - a: largest integer such that the cumulative binomial sum is less than 2^threshold_power
    """
    threshold = 2^threshold_power
    running_sum = 0
    a = 0

    while True:
        running_sum += binomial(n, a)
        if running_sum >= threshold:
            return a - 1
        a += 1

# Example usage:
assert largest_a_below_binom_sum(256, 128) == 29

In [12]:
from rok_estimator.rok_estimator import *


def RelationRPWithKJL(ratio = 18, height = 105, secpar=110):
    class JL:
        kjl = ratio
        njl = height
        mjl = kjl * njl
        alpha_inf = largest_a_below_binom_sum(njl,secpar)
        
        def get_delta(self):
            return 2^(-secpar)
    
        def get_right_bound(self):
            return sqrt(self.mjl * self.njl)
            
        def get_left_bound(self):
            return self.alpha_inf / (2 * sqrt(self.mjl))
    
        def get_right_bound_inf(self):
            return self.mjl
            
        def get_left_bound_inf(self):
            return self.alpha_inf / (2)
            

    # class HJL: 
    #     kjl = ratio
    #     njl = 120
    #     mjl = kjl * njl
    #     alpha_inf = 42 
    
    #     def get_delta(self):
    #         return 2^(-110)
    
    #     def get_right_bound(self):
    #         return sqrt(207)
            
    #     def get_left_bound(self):
    #         return sqrt(7)
    
    #     def get_right_bound_inf(self):
    #         return self.mjl
            
    #     def get_left_bound_inf(self):
    #         return self.alpha_inf / (2)
    
    
    
    
    jl = JL()
    
    print("Limits of structure: ", jl.mjl)
    
    
    g_first_branch = None
    g_second_branch = None
    
    g_second_branch_ext_2_norm = None
    g_second_branch_ext_inf_norm = None
    
    class Relation_RP(Relation):
        def execute(self, op, **kwargs):
            match op:
                case "srp":
                    return self.pi_srp(**kwargs)
                case "to-first":
                    return self.pi_to_first(**kwargs)
                case "join":
                    return self.pi_join(**kwargs)
                case "norm-poly":
                    return self.pi_norm_poly()
                case _:
                    return super().execute(op, **kwargs)
                
        def pi_norm_poly(self):
            comm = (3 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.size_Rq()  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            # comm = (2 * self.rep) * self.ring.size_Rq() + (1 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.log_q  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            snd_err = 3 * ceil(log(self.wdim, 2))/(2**(self.ring.log_q * self.ring.residue_deg)) # TODO: Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the soundness error.
            rel_params = {
                # "ring": self.ring,
                # "trivial": self.trivial,
                "op_name": "norm-poly",
                "n_compress": self.n_compress + 2,
                # "n_commit": self.n_commit,
                "n_rel": self.n_rel + 2,
                # "wdim": self.wdim,
                # "rep": self.rep,
                # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
                # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
                # "log_beta_wit_inf": self.log_beta_wit_inf
                "comm" : comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,           
                "log_beta_ext_2_func" : lambda x : self.log_beta_wit_2, # perfect extraction
                "log_beta_ext_inf_func" : lambda x : oo # extraction of ell_2-norm is perfect, then bound ell_inf-norm by norm conversion
            }      
            return replace(self, **rel_params)
                
        def pi_srp(self):
            global g_first_branch 
            rel_params = {
                "n_compress": self.n_commit + 1,
                "n_rel": self.n_rel + 1,
                "log_slack_2_func": lambda x: x,
                "log_slack_inf_func": lambda x: x, 
            }
    
            g_first_branch = replace(deepcopy(self), **rel_params)
            comm = (self.n_commit + 1) * self.ring.size_Rq()  # for the new comm
            snd_err = self.wdim * self.rep / jl.kjl * (1 / 2**(self.ring.log_q * self.ring.residue_deg) + jl.get_delta() * euler_phi(self.ring.f) / jl.njl)
            global g_second_branch_ext_2_norm
            global g_second_branch_ext_inf_norm
    
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                return g_second_branch_ext_2_norm + log(1 / jl.get_left_bound(), 2) + log(sqrt(radical(self.ring.f)),2)
                
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm 
                return g_second_branch_ext_inf_norm + log(1 / jl.get_left_bound_inf(), 2)
                
            rel_params = {
                "op_name": "srp 2 rel",
                "n_compress": self.n_commit + 1,
                "n_rel": 1,
                "wdim": ZZ(ceil(self.wdim / jl.kjl * self.rep)),
                "rep": 1,
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,    
                "log_beta_wit_2": self.log_beta_wit_2 + log(jl.get_right_bound(), 2) + log(sqrt(self.rep),2),
                "log_beta_wit_inf":self.log_beta_wit_inf + log(jl.get_right_bound_inf(), 2) + log(sqrt(self.rep),2), # can we say sth about infinity - norm?
                "log_beta_ext_2_func": log_beta_ext_2_func, # we extract norm from the second branch (relax it)
                "log_beta_ext_inf_func" : log_beta_ext_inf_func,  # can we say sth about infinity - norm?
                "log_slack_2_func": lambda x: 0,
                "log_slack_inf_func": lambda x: 0,
            }     
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)
    
    
        def pi_to_first(self):
            global g_second_branch 
            g_second_branch = deepcopy(self)
            
            rel_params = {
                "op_name": "    1 rel",
                "n_rel": 1,
                "log_beta_ext_2_func" : lambda x, _ : x, # perfect extraction
                "log_beta_ext_inf_func" : lambda x, _ : x, # perfect extraction
                "acc_comm": g_second_branch.acc_comm,
                "comm": 0,
                "snd_err" : g_second_branch.snd_err,
                "acc_snd_err" : g_second_branch.acc_snd_err,    
                # "log_slack_2_func" : lambda x: 0, # perfect extraction
                # "log_slack_inf_func" : lambda x: 0 # perfect extraction
            }      
            global g_first_branch
            return replace(g_first_branch, **rel_params)
            
        def pi_join(self):
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                g_second_branch_ext_2_norm = x
                return x
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm
                g_second_branch_ext_inf_norm = x
                return x
    
            global g_second_branch
    
            assert g_second_branch.wdim <= self.wdim
            assert g_second_branch.rep == 1
            assert self.rep == 1

            comm = (self.n_rel + g_second_branch.n_rel) * self.ring.size_Rq()  # for the new comm
            
            rel_params = {
                "op_name": "join",
                "rep": 2,
                "n_rel": self.n_rel + g_second_branch.n_rel,
                "n_compress": self.n_compress + g_second_branch.n_compress,
                "comm" : comm,      
                "acc_comm" : self.acc_comm + comm,  
                "log_beta_wit_2": max(self.log_beta_wit_2, g_second_branch.log_beta_wit_2),
                "log_beta_wit_inf": max(self.log_beta_wit_inf, g_second_branch.log_beta_wit_inf),
                "log_beta_ext_2_func": log_beta_ext_2_func, # perfect extraction, but we keep track of what was there
                "log_beta_ext_inf_func" : log_beta_ext_inf_func # perfect extraction, but we keep track of what was there
            }      
            return replace(self, **rel_params)
    return Relation_RP
        

def challenge_set(max_v = 100,  f = 60):
    K = CyclotomicField(f)
    phi = euler_phi(f)
    R = K.ring_of_integers()
    
    
    return ChallengeSet(cardinality = max_v^phi, gamma_2 = max_v * sqrt(phi) / 2, theta_2 = max_v * sqrt(phi), gamma_inf = max_v / 2, theta_inf = max_v)

In [10]:
# A 80

log_beta_sis_2 = 37.8

ring_params = {
    "f": 60,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": 100,
    "secpar_target": 80,
    "kappa_target": 80
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 split      |   10747904 |  25 |    ( 31 | 34/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |      (107.8 KB | 107.8 KB)      |         (2^-95  | 2^-95 )         
 bdecomp    |   10747904 | 100 |    ( 19 | 23/  0 )        |     (  3 | 23/  0 )         | 8200. MB   |      (336.9 KB | 444.7 KB)      |         (2^-oo  | 2^-95 )         
 norm       |   10747904 | 109 |    ( 19 | 37/  0 )        |     (  3 | 37/  0 )         | 8938. MB   |      (104.3 KB | 549.0 KB)      |         (2^-96  | 2^-94 )         
 batch      |   10747904 | 109 |    ( 19 | 37/  0 )        |     (  3 | 37/  0 )         | 8938. MB   |      (0.0000 B

In [15]:
# A NEW

ring_params = {
    "f": 60,
    "log_beta_sis_2": 55,
    "log_q": 100,
    "residue_deg": 5,
    "secpar_target": 80,
    "kappa_target": 80
    
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 14+ [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 106, 106))
sim.ring.C = challenge_set(100)


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  1908
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^100, beta_sis_2: 2^55
    SIS module rank n_sis: 47, target SIS security: 80, resulting SIS security: 80
    residue degree: 5, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: None
    |R_q| = 200.0 B, |R_q^(n_sis)| = 9.180 KB
 
Challenge set parameters:
    cardinality: 100000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 33/  0 )        |     ( 14 | 33/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp    

In [275]:
# A 128

ring_params = {
    "f": 60,
    "log_beta_sis_2": 38.6,
    "log_q": 100,
    "secpar_target": 128,
    "kappa_target": 128
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 split      |   10747904 |  25 |    ( 31 | 34/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |      (173.4 KB | 173.4 KB)      |         (2^-195 | 2^-195)         
 bdecomp    |   10747904 | 100 |    ( 19 | 23/  0 )        |     (  3 | 23/  0 )         | 8200. MB   |      (542.0 KB | 715.4 KB)      |         (2^-oo  | 2^-195)         
 norm       |   10747904 | 109 |    ( 19 | 38/  0 )        |     (  3 | 37/  0 )         | 8938. MB   |      (128.9 KB | 844.3 KB)      |         (2^-196 | 2^-194)         
 batch      |   10747904 | 109 |    ( 19 | 38/  0 )        |     (  3 | 37/  0 )         | 8938. MB   |      (0.0000 B

In [291]:
# A NEW 128

ring_params = {
    "f": 60,
    "log_beta_sis_2": 57.8,
    "log_q": 100,
    "residue_deg": 6,
    "secpar_target": 128,
    "kappa_target": 128
    
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 14+ [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 153, 153))
sim.ring.C = challenge_set(400)


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2754
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^100, beta_sis_2: 2^57.8000000000000
    SIS module rank n_sis: 82, target SIS security: 128, resulting SIS security: 129
    residue degree: 6, target Schwartz-Zippel security: 128, resulting Schwartz-Zippel security: None
    |R_q| = 200.0 B, |R_q^(n_sis)| = 16.02 KB
 
Challenge set parameters:
    cardinality: 429496729600000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^11
    forward ell_inf expansion factor gamma_inf: 2^8
    inverse ell_inf expansion factor theta_inf: 2^9
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 33/  0 )        |     ( 14 | 33/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2

In [377]:
# A 196

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": 44.9,
    "log_q": 100,
    "secpar_target": 196,
    "kappa_target": 196,
    "residue_deg": 3
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  134348800 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 35/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 split      |    5373952 |  25 |    ( 31 | 35/  0 )        |     ( 14 | 35/  0 )         | 7688. MB   |      (337.5 KB | 337.5 KB)      |         (2^-295 | 2^-295)         
 bdecomp    |    5373952 | 100 |    ( 20 | 23/  0 )        |     (  3 | 23/  0 )         | 8200. MB   |      (1.030 MB | 1.360 MB)      |         (2^-oo  | 2^-295)         
 norm       |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938. MB   |      (254.3 KB | 1.608 MB)      |         (2^-296 | 2^-294)         
 batch      |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938. MB   |      (0.0000 B

In [349]:
# A NEW 196

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": 57.5,
    "log_q": 100,
    "residue_deg": 8,
    "secpar_target": 196,
    "kappa_target": 196
    
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 14+ [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 226, 226))
sim.ring.C = challenge_set(83,ring_params["f"])


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  4068
Ring parameters:
    conductor f: 120, degree phi: 32, modulus q: 2^100, beta_sis_2: 2^57.5000000000000
    SIS module rank n_sis: 59, target SIS security: 196, resulting SIS security: 197
    residue degree: 8, target Schwartz-Zippel security: 196, resulting Schwartz-Zippel security: None
    |R_q| = 400.0 B, |R_q^(n_sis)| = 23.05 KB
 
Challenge set parameters:
    cardinality: 25733505785478367615107164358252046214490685371339768616901761
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  134348800 |   1 |    ( 33 | 34/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      | 

In [378]:
# A 256

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": 45.5,
    "log_q": 64,
    "secpar_target": 256,
    "kappa_target": 256
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  134348800 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 35/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 split      |    5373952 |  25 |    ( 31 | 35/  0 )        |     ( 14 | 35/  0 )         | 7688. MB   |      (438.0 KB | 438.0 KB)      |         (2^-315 | 2^-315)         
 bdecomp    |    5373952 | 100 |    ( 20 | 23/  0 )        |     (  3 | 23/  0 )         | 8200. MB   |      (1.337 MB | 1.764 MB)      |         (2^-oo  | 2^-315)         
 norm       |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938. MB   |      (246.0 KB | 2.005 MB)      |         (2^-316 | 2^-314)         
 batch      |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938. MB   |      (0.0000 B

In [16]:
### A NEW 256

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": 60.6,
    "log_q": 100,
    "secpar_target": 256,
    "kappa_target": 256,
    "residue_deg": 5
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 13+ [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 280, 280))
sim.ring.C = challenge_set(639, ring_params["f"])


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  5040
Ring parameters:
    conductor f: 120, degree phi: 32, modulus q: 2^100, beta_sis_2: 2^60.6000000000000
    SIS module rank n_sis: 82, target SIS security: 256, resulting SIS security: 256
    residue degree: 5, target Schwartz-Zippel security: 256, resulting Schwartz-Zippel security: None
    |R_q| = 400.0 B, |R_q^(n_sis)| = 32.03 KB
 
Challenge set parameters:
    cardinality: 597073039182879857989553168123669664326517795218001082090332702878587253341837810932101121
    forward ell_2 expansion factor gamma_2: 2^11
    inverse ell_2 expansion factor theta_2: 2^12
    forward ell_inf expansion factor gamma_inf: 2^9
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  134348800 |   1 |    ( 33 | 34/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |    

In [6]:
# B NEW

ring_params = {
    "f": 60,
    "log_beta_sis_2": 73.6,
    "log_q": 75.6,
}

rel_params = {
    "wdim": 2**20 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(2) * 16 + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(20, 108))
sim.ring.C = C

print(sim.ring)
sim.execute(ops)
sim.extract()
sim.show()

Limits of structure:  2160
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^75.6000000000000, beta_sis_2: 2^73.6000000000000
    SIS module rank n_sis: 174, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 151.200000000000
    |R_q| = 151.2 B, |R_q^(n_sis)| = 25.69 KB
 
Challenge set parameters:
    cardinality: 100000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       | 1074790400 |   1 |    ( 34 | 34/  0 )        |     ( 14 | 34/  0 )         | 30750. MB  |      (0.0000 B | 0.0000 B)      |      

In [105]:
# C

ring_params = {
    "f": 60,
    "log_beta_sis_2": 39.8,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**22 * 41,
    "rep": 25,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = 16 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  171966464 |  25 |    ( 33 | 36/  0 )        |     ( 14 | 36/  0 )         | 123000. MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp    |  171966464 | 100 |    ( 21 | 25/  0 )        |     (  3 | 25/  0 )         | 131200. MB |      (571.9 KB | 571.9 KB)      |         (2^-oo  | 2^-oo )         
 norm       |  171966464 | 110 |    ( 21 | 39/  0 )        |     (  3 | 39/  0 )         | 144300. MB |      (117.5 KB | 689.4 KB)      |         (2^-124 | 2^-124)         
 batch      |  171966464 | 110 |    ( 21 | 39/  0 )        |     (  3 | 39/  0 )         | 144300. MB |      (0.0000 B | 689.4 KB)      |         (2^-119 | 2^-119)         
 split      |   85983232 | 220 |    ( 21 | 39/  0 )        |     (  3 | 39/  0 )         | 144300. MB |      (880.0 KB

In [7]:
# C NEW

ring_params = {
    "f": 60,
    "log_beta_sis_2": 74.7,
    "log_q": 76.9,
}

rel_params = {
    "wdim": 2**22 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(2) * 18 + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(20, 110))
sim.ring.C = C


print(sim.ring)
sim.execute(ops)
sim.extract()
sim.show()

Limits of structure:  2200
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^76.9000000000000, beta_sis_2: 2^74.7000000000000
    SIS module rank n_sis: 176, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 153.800000000000
    |R_q| = 153.8 B, |R_q^(n_sis)| = 26.43 KB
 
Challenge set parameters:
    cardinality: 100000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       | 4299161600 |   1 |    ( 35 | 35/  0 )        |     ( 14 | 35/  0 )         | 123000. MB |      (0.0000 B | 0.0000 B)      |      